# Local reliability: evaluate

Evaluate local reliability for 3 uncertain parameters (fimp_14, psepbqarmax and triang). The intention is to try out the normalised severity function (relative to the solution severity); is the sample point more or less violated than the solution point?

In [1]:
import easyvvuq as uq
import chaospy as cp
from pathlib import Path
from infeas.decoder import MfileDecoder

In [2]:
# Define campaign
WORK_DIR = "campaigns"
Path("campaigns").mkdir(exist_ok=True)
campaign = uq.Campaign(name="3p_infeas_sol", work_dir=WORK_DIR)

# Define parameter space
# Uncertainties from Alex's SA paper

params = {
    # Default values are at solution point
    "fimp_14": {
        "type": "float",
        # TODO
        "min": 5.0e-6,
        "max": 1.0e-4,
        "default": 5.0e-6,
    },  # ok
    "psepbqarmax": {"type": "float", "min": 8.7, "max": 10.0, "default": 10.0},  # ok
    "triang": {"type": "float", "min": 0.4, "max": 0.6, "default": 0.5},  # ok
}

# QoIs
# Violated constraint residuals
qois = [
    "rms_vio_constr_res",
]

Next, define a custom decoder to handle Process's mfile output.

In [3]:
# Create encoder and decoder
encoder = uq.encoders.GenericEncoder(
    template_fname="large_tokamak_sol_IN_DAT.template", target_filename="IN.DAT"
)

decoder = MfileDecoder(target_filename="MFILE.DAT", output_columns=qois)

cmd = "process -i IN.DAT"
actions = uq.actions.local_execute(encoder, cmd, decoder)

# Add the app
campaign.add_app(name="feasibility", params=params, actions=actions)

In [4]:
# Create PCE sampler
# Vary 3 uncertain inputs

# 1p: triang
# 2p: triang, psepbqarmax
# 3p: triang, psepbqarmax, fimp_14

vary = {
    "fimp_14": cp.Uniform(
        # Original
        # 1.0e-6,
        # 1.0e-4,
        # Modified to include soln
        5.0e-6,
        1.0e-5,
    ),
    "psepbqarmax": cp.Uniform(
        8.7,
        10.0,
    ),
    "triang": cp.Uniform(
        0.4,
        0.6,
    ),
}
# sampler = uq.sampling.PCESampler(vary=vary, polynomial_order=4)
# Optional Saltelli MC
sampler = uq.sampling.MCSampler(vary=vary, n_mc_samples=30)
# Generates a Saltelli sampling plan of n_mc*(n_params + 2) input samples

# Add sampler to campaign
campaign.set_sampler(sampler)

# Draw samples, execute and collate
campaign.execute().collate(progress_bar=True)
samples = campaign.get_collation_result()

  0%|          | 0/90 [00:00<?, ?it/s]

The IN.DAT file does not contain any obsolete variables.
 The total number of constraints is counted automatically and does not need to be stated in IN.DAT.
 tmargmin_cs and tmargmin should not both be specified in IN.DAT.
 tmargmin_cs has been ignored.
 
 **************************************************************************************************************
 ************************************************** PROCESS ***************************************************
 ************************************** Power Reactor Optimisation Code ***************************************
 **************************************************************************************************************
 
   Program :
   Version : 3.0.2   Release Date :: 2024-01-25
   Tag No. : v3.0.2-32-g2958d5e
    Branch : main
   Git log : Add tests for parsing real inputs in IN.DAT
 Date/time : 29 Feb 2024 12:48:10 +00:00(hh:mm) UTC
      User : jon
  Computer : jon-Precision-3560
 Directory :
 Warning in 

process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
  1%|          | 1/90 [00:02<03:52,  2.61s/it]

The IN.DAT file does not contain any obsolete variables. 
 ******************************************** Errors and Warnings *********************************************
 
 PROCESS status flag:   Warning messages                                  
 
 ID  LEVEL  MESSAGE
244     2   PHYSICS: Diamagnetic fraction is more than 1%, but not calculated. Consider usin
  
 
 ******************************************* End of PROCESS Output ********************************************
 

 The total number of constraints is counted automatically and does not need to be stated in IN.DAT.
 tmargmin_cs and tmargmin should not both be specified in IN.DAT.
 tmargmin_cs has been ignored.
The IN.DAT file does not contain any obsolete variables.
 
 **************************************************************************************************************
 ************************************************** PROCESS ***************************************************
 *************************************

process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
process.power - WARNING - WARNING: Power balance across separatrix is in error by more than 5 MW.
  2%|▏         | 2/90 [00:03<02:16,  1.55s/it]

...

 98%|█████████▊| 88/90 [00:42<00:00,  4.00it/s]

 Warning in routine OCMMNT :
Triple product = Vol-average electron density x Vol-average         & electron temperature x Energy confinement time:
This is longer than 110 columns.
 
 ******************************************** Errors and Warnings *********************************************
 
 PROCESS status flag:   Warning messages                                  
 
 ID  LEVEL  MESSAGE
243     2   PHYSICS: Predicted plasma driven current is more than upper limit on non-inducti
  
244     2   PHYSICS: Diamagnetic fraction is more than 1%, but not calculated. Consider usin
  
 
 ******************************************* End of PROCESS Output ********************************************
 


 99%|█████████▉| 89/90 [00:43<00:00,  3.33it/s]

 Warning in routine OCMMNT :
Triple product = Vol-average electron density x Vol-average         & electron temperature x Energy confinement time:
This is longer than 110 columns.
 
 ******************************************** Errors and Warnings *********************************************
 
 PROCESS status flag:   Warning messages                                  
 
 ID  LEVEL  MESSAGE
243     2   PHYSICS: Predicted plasma driven current is more than upper limit on non-inducti
  
244     2   PHYSICS: Diamagnetic fraction is more than 1%, but not calculated. Consider usin
  
 
 ******************************************* End of PROCESS Output ********************************************
 


100%|██████████| 90/90 [00:43<00:00,  2.08it/s]
